
# ORE Jupyter Dashboard

This dashboard contains several modules to run ORE functionality and visualize results.

It assumes a standard installation of Anaconda Python 3, see https://www.continuum.io/downloads. 

Some plots require more dependencies, see below.

## Launch ORE

Kick off a process in ORE as specified in Input/ore.xml

In [ ]:
from OREAnalytics import *

print ("Loading parameters...")
params = Parameters()
print ("   params is of type", type(params))
params.fromFile("Input/ore.xml")
print ("   setup/asofdate = " + params.get("setup","asofDate"))

print ("Building OREApp...")
ore = OREApp(params)
print ("   ore is of type", type(ore))

In [ ]:
print ("Running ORE process...");
ore.buildMarket()
print ("ORE process finished");

## Query ORE App Members

Retrieve the market object stored in OREApp and initialized in the previous step when running the ORE process.

Then query the market object for some members (a discount and a forward curve) and evaluate discount factors and zero rates at some point in time in the future.

In [ ]:
print ("Retrieve market object...");
market = ore.getMarket()
print ("   retrieved market is of type", type(market))
asof = market.asofDate();
print ("   retrieved market's asof date is", asof)

In [ ]:
ccy = "EUR"
index = "EUR-EURIBOR-6M"

print ("Get term structures for ccy ", ccy, "and index", index);

discountCurve = market.discountCurve(ccy)
print ("   discount curve is of type", type(discountCurve))

iborIndex = market.iborIndex(index)
print ("   ibor index is of type", type(iborIndex))

forwardCurve = iborIndex.forwardingTermStructure()
print ("   forward curve is of type", type(forwardCurve))

date = asof + 10*Years;
zeroRateDc = Actual365Fixed()

discount = discountCurve.discount(date)
zero = discountCurve.zeroRate(date, zeroRateDc, Continuous)

fwdDiscount = forwardCurve.discount(date)
fwdZero = forwardCurve.zeroRate(date, zeroRateDc, Continuous)

print ("   10y discount factor (discount curve) is", discount)
print ("   10y discout factor (forward curve) is", fwdDiscount)

print ("   10y zero rate (discount curve) is", zero)
print ("   10y zero rate (forward curve) is", fwdZero)

In [ ]:
dc = ActualActual()

# date grid
dates = []
times = []
zeros1 = []
zeros2 = []
date = asof
previousDate = asof
for i in range (1,10*53):
    date = date + Period(1, Weeks);
    time = dc.yearFraction(asof, date)
    dates.append(date)
    times.append(time)
    zero1 = discountCurve.forwardRate(previousDate, date, zeroRateDc, Continuous).rate()
    zero2 = forwardCurve.forwardRate(previousDate, date, zeroRateDc, Continuous).rate()
    zeros1.append(zero1)
    zeros2.append(zero2)
    previousDate = date
    #print (date, time, zero1, zero2)

#print(times)

## Curve Plot

Plot the discount and forward curves above using Bloomberg's bqplot, see https://github.com/bloomberg/bqplot

In [ ]:
import numpy as np
import bqplot as bq
from traitlets import link
from IPython.display import display

xs = bq.LinearScale()
ys = bq.LinearScale()
x = times
y = np.vstack((zeros1,zeros2))

line = bq.Lines(x=x, y=y, scales={'x': xs, 'y': ys}, display_legend=True, labels=['Discount Curve','Forward Curve'])
xax = bq.Axis(scale=xs, label='years', grid_lines='solid')
yax = bq.Axis(scale=ys, orientation='vertical', grid_lines='solid')

fig = bq.Figure(marks=[line], axes=[xax, yax], animation_duration=1000, legend_location='top-left')
display(fig)